# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [4]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [6]:
data2 = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum()
data2=pd.DataFrame(data2)
data2.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [7]:
table = data.pivot_table(values='Quantity', index=['ProductName'], columns=['CustomerID']
                    , aggfunc='sum').fillna(0)
table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
distance = squareform(pdist(table.T, 'euclidean'))
distance

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
customer = 200
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

#distances
#top5 = distances['200'].sort()
#top5 = distances.nlargest(6, customer)
#top5 = distances.sort_values(distances[customer]).tail()
top5 = distances[customer].sort_values(ascending=False)[1:6]

#top5 = distances.nlargest(6, [customer])[1:6]

top5

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
top_purchases = data2.loc[list(top5.index)]
top_purchases = pd.DataFrame(top_purchases)
top_purchases.head()

Quantity
CustomerID ProductName                                
1072       Appetizer - Mini Egg Roll, Shrimp         1
           Appetizer - Sausage Rolls                 1
           Beans - Kidney, Canned                    1
           Beef - Montreal Smoked Brisket            1
           Beef - Striploin Aa                       1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [11]:
top= top_purchases.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity', ascending = False)
top.head()

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [12]:
u264 = table[table[264]==0]
merged= u264.merge(top, left_on='ProductName', right_on='ProductName').sort_values('Quantity', ascending= False)
u264= merged[264].head()
u264


ProductName
Lamb - Ground            0.0
Pork - Kidney            0.0
Pepper - Black, Whole    0.0
Bay Leaf                 0.0
Wanton Wrap              0.0
Name: 264, dtype: float64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
recommendations = {}
customers = list(set(data['CustomerID']))
for customer in customers:
    top5 = distances[customer].sort_values(ascending=False)[1:6]
    top_purchases = data.loc[list(top5.index)]
    top_products = top_purchases.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
    customer_prods = top.loc[:, customer]
    customer_prods = list(customer_prods[customer_prods!=0].index)
    not_purchased = list(top_products.loc[~top_products.index.isin(customer_prods)][:5].index)
    recommendations[customer] = not_purchased
    

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [83973] of <class 'int'>

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
dfrecommendations = pd.DataFrame(recommendations).T.reset_index()
cols = ['CustomerID', '0', '1', '2', '3', '4']
dfrecommendations.columns = cols
dfrecommendations.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 6 elements

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [ ]:
def recommendations(data, distance_type):
    recommendations = {}
    data['SalesID'] = data['SalesID'].astype(str)
    data['ProductID'] = data['ProductID'].astype(str)
    df = data.groupby(['CustomerID', 'ProductName']).sum()
    prod_cust = pd.pivot_table(data=df, index='ProductName', columns='CustomerID', values='Quantity', aggfunc=sum).fillna(0)
    cust_similarity = squareform(pdist(prod_cust.T, distance_type))
    distances = pd.DataFrame(1/(1 + cust_similarity), index=prod_cust.columns, columns=prod_cust.columns)
    customers = list(set(data['CustomerID']))
    for customer in customers:
        top5 = distances[customer].sort_values(ascending=False)[1:6]
        top5_purchases = df.loc[list(top5.index)]
        top5_products = top5_purchases.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
        customer_prods = prod_cust.loc[:, customer]
        customer_prods = list(customer_prods[customer_prods!=0].index)
        not_purchased = list(top5_products.loc[~top5_products.index.isin(customer_prods)][:5].index)
        recommendations[customer] = not_purchased
    dfrecommendations = pd.DataFrame(recommendations).T.reset_index()
    return dfrecommendations

distance_types = ['euclidean', 'correlation', 'cityblock', 'cosine', 'jaccard']
for distancetype in distance_types:
    print(distancetype)
    display(recommendations(data, distancetype)[:1])